In [1]:
import pandas as pd

In [3]:
import os
import sys

current_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, ".."))
sys.path.insert(0, parent_dir)

from src.orcid_data_utilities import get_author_info_bicocca, search_orcid_bicocca

In [6]:
df = pd.read_csv("../data/authors/authors_data_science.csv")

# Quelli certi con orcid trovato

In [7]:
for index, row in df.iterrows():
        family_name = row['Last name']
        given_name = row['Name']
        orcid = search_orcid_bicocca(given_name, family_name)
        #if not orcid:
        #        orcid = search_orcid_all_matches(given_name, family_name)
        df.loc[index, "orcid"] = orcid

Nessun ORCID trovato associato alla Bicocca.
Trovato: Simone  -> ORCID: 0000-0002-7070-1545
Trovato: Pier Giovanni  -> ORCID: 0000-0003-3769-6649
Nessun ORCID trovato associato alla Bicocca.
Trovato: Davide  -> ORCID: 0000-0001-9655-7142
Trovato: Michele  -> ORCID: 0000-0002-2480-966X
Trovato: Gianluca  -> ORCID: 0000-0001-5584-3089
Nessun ORCID trovato associato alla Bicocca.
Nessun ORCID trovato associato alla Bicocca.
Nessun ORCID trovato associato alla Bicocca.
Trovato: Gianfranco  -> ORCID: 0000-0002-3412-4162
Trovato: Gabriele  -> ORCID: 0000-0001-5186-0199
Trovato: marco  -> ORCID: 0000-0001-7415-0771
Trovato: Andrea  -> ORCID: 0000-0001-9803-3668
Trovato: Fabio  -> ORCID: 0000-0001-6864-2702
Trovato: Enza  -> ORCID: 0000-0002-4062-0824
Trovato: gianna  -> ORCID: 0000-0002-7952-3618
Trovato: Enrico  -> ORCID: 0000-0002-3436-1395
Trovato: Paolo  -> ORCID: 0000-0001-9112-0574
Trovato: Marco  -> ORCID: 0000-0003-2461-275X
Trovato: MATTEO  -> ORCID: 0000-0002-1801-5118
Trovato: Gabr

In [8]:
df_null = df[df["orcid"].isna()]
df_null.to_csv("../data/authors/authors_null.csv", index=False)

In [9]:
df

,Last name,Name,SSD,orcid
0,Bernasconi,Davide Paolo,MED/01,None
1,Bianco,Simone,INF/01,0000-0002-7070-1545
2,Bissiri,Pier Giovanni,SECS-S/01,0000-0003-3769-6649
3,Cesarini,Mirko,ING-INF/05,None
4,Chicco,Davide,INF/01,0000-0001-9655-7142
5,Ciavotta,Michele,INF/01,0000-0002-2480-966X
6,Della Vedova,Gianluca,INF/01,0000-0001-5584-3089
7,Fattore,Marco,SECS-S/03,None
8,Ferretti,Claudio,INF/01,None
9,Fersini,Elisabetta,INF/01,None


In [10]:
df.to_csv("../data/authors/authors_complete.csv")

# Openalex

In [11]:
df = pd.read_csv('../data/authors/authors_complete.csv', index_col=False)

for index, row in df.iterrows():
    family_name = row['Last name']
    given_name = row['Name']
    res = get_author_info_bicocca(f"{given_name} {family_name}")
    if res:
        df.loc[index, "hindex"] = int(res["hindex"])
        df.loc[index, "openalex id"] = res["id"]
        df.loc[index, "institutions id"] = str(res["ins_id"])
        df.loc[index, "institutions name"] = str(res["ins_name"])
        df.loc[index, "topics"] = str(res["topics"])
        if pd.notna(row['orcid']) and row['orcid'].strip() != "":
            orcid_value = row['orcid']

df.to_csv("../data/authors/authors_complete.csv", index=False)


# Non trovati

In [ ]:
def search_orcid_all_matches(given_name, family_name):
    base_url = "https://pub.orcid.org/v3.0/expanded-search/"
    query = f"given-names:{given_name} AND family-name:{family_name}"

    headers = {
        "Accept": "application/json"
    }

    params = {
        "q": query
    }

    response = requests.get(base_url, headers=headers, params=params)

    if response.status_code != 200:
        print(f"Errore nella richiesta: {response.status_code}")
        return None

    results = response.json().get("expanded-result", [])

    if not results:
        print("Nessun risultato trovato.")
        return None

    for res in results:
        full_name = res.get("given-names", "") + " " + res.get("family-name", "")
        orcid = res.get("orcid-id", "N/A")
        institutions = res.get("institution-name", [])
        print(f"{full_name} -> ORCID: {orcid} | Institutions: {institutions}")

    return orcid

In [14]:
for index, row in df.iterrows():
        family_name = row['Last name']
        given_name = row['Name']
        orcid = search_orcid_bicocca(given_name, family_name)
        #if not orcid:
        #        orcid = search_orcid_all_matches(given_name, family_name)
        df.loc[index, "orcid"] = orcid

Nessun risultato trovato.
Nessun ORCID trovato associato alla Bicocca.
Trovato: Simone  -> ORCID: 0000-0002-7070-1545
Trovato: Pier Giovanni  -> ORCID: 0000-0003-3769-6649
Nessun ORCID trovato associato alla Bicocca.
Trovato: Davide  -> ORCID: 0000-0001-9655-7142
Trovato: Michele  -> ORCID: 0000-0002-2480-966X
Trovato: Gianluca  -> ORCID: 0000-0001-5584-3089
Nessun ORCID trovato associato alla Bicocca.
Nessun ORCID trovato associato alla Bicocca.
Nessun ORCID trovato associato alla Bicocca.
Nessun ORCID trovato associato alla Bicocca.
Trovato: Gianfranco  -> ORCID: 0000-0002-3412-4162
Trovato: Gabriele  -> ORCID: 0000-0001-5186-0199
Trovato: marco  -> ORCID: 0000-0001-7415-0771
Trovato: Andrea  -> ORCID: 0000-0001-9803-3668
Trovato: Fabio  -> ORCID: 0000-0001-6864-2702
Trovato: Enza  -> ORCID: 0000-0002-4062-0824
Trovato: gianna  -> ORCID: 0000-0002-7952-3618
Trovato: Enrico  -> ORCID: 0000-0002-3436-1395
Trovato: Paolo  -> ORCID: 0000-0001-9112-0574
Trovato: Marco  -> ORCID: 0000-0003

In [12]:
df

,Last name,Name,SSD,orcid
0,Andreotti,Alberta Argia,SPS/09,None
1,Bernasconi,Davide Paolo,MED/01,0000-0001-6043-3367
2,Bianco,Simone,INF/01,0000-0002-7070-1545
3,Bissiri,Pier Giovanni,SECS-S/01,0000-0003-3769-6649
4,Cesarini,Mirko,ING-INF/05,0000-0001-9601-0403
5,Chicco,Davide,INF/01,0000-0001-9655-7142
6,Ciavotta,Michele,INF/01,0000-0002-2480-966X
7,Della Vedova,Gianluca,INF/01,0000-0001-5584-3089
8,Di Domenica,Nico,SECS-P/08,0009-0001-4949-0624
9,Fattore,Marco,SECS-S/03,0000-0002-6624-8637
